In [ ]:
!pip install boto3
!pip install torch transformers diffgram neo4j anthropic pandas tqdm
!pip install llama_index
!pip install matplotlib
!pip install tabulate
!pip install 'whylogs[viz]'

In [1]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from diffgram import Project
from typing import List, Dict, Optional
import anthropic
import json
from neo4j import GraphDatabase
from tqdm import tqdm
import logging
import os
import sys
import boto3
import requests
import pprint
import json
from diffgram import Project
from llama_index.core import SimpleDirectoryReader, StorageContext
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib as mpl
from tabulate import tabulate
import math

/tmp/ipykernel_75/3258265333.py:21: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [2]:
# Use os.getcwd() since __file__ is not available in interactive environments
current_dir = os.getcwd()

# If your structure is such that the package is in the parent directory, compute the parent directory:
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [3]:
import diffgram_utils as du

## Connect  to Diffgram

In [4]:
# Configuration
DIFFGRAM_CONFIG = {
    "host": "http://dispatcher:8085",
    "project_string_id": "translucenttracker",
    "client_id": "LIVE__u3v8q0m7tx1p851dp0ap",
    "client_secret": "1qgd8as7xfcbuem6mw9j1z0xvjfmmvlagbugqr8z1g1ntypugr2ul24cce5k"
}

In [5]:
project = Project(host=DIFFGRAM_CONFIG["host"],
        project_string_id = "translucenttracker",
        client_id = "LIVE__u3v8q0m7tx1p851dp0ap",
        client_secret = "1qgd8as7xfcbuem6mw9j1z0xvjfmmvlagbugqr8z1g1ntypugr2ul24cce5k"
      )
project_local = project
auth = project.session.auth

In [6]:
# Define constants
BATCH_SIZE = 32
MAX_LENGTH = 256
NUM_TRAIN_SAMPLES = 5440  # Number of samples to use for training
NUM_TRAINING_DATA = 5440
train_dataset_suffix = "NER_train_batch_"
test_dataset_suffix = "NER_test_batch_"
JOB_NAME = "Law_NER_task1"
JOB_TRAIN_SUFFIX = "NER_train_JOB_"
JOB_TEST_SUFFIX = "NER_test_JOB_"
MAX_NUM_OF_TASK = 250
NER_schema_name = 'ENTITY_TRAINING_SCHEMA'

## Get  schema ID

In [7]:
schema_id = du.find_schema(NER_schema_name, project)

Existing Schemas in Diffgram:
[
  {
    "archived": false,
    "id": 8,
    "is_default": true,
    "member_created_id": 1,
    "member_updated_id": null,
    "name": "Default Schema",
    "project_id": 4,
    "time_created": "2025-02-04 22:16:17",
    "time_updated": null
  },
  {
    "archived": false,
    "id": 9,
    "is_default": false,
    "member_created_id": 10,
    "member_updated_id": null,
    "name": "NER_TRAINING_SCHEMA",
    "project_id": 4,
    "time_created": "2025-02-05 17:08:24",
    "time_updated": null
  },
  {
    "archived": false,
    "id": 11,
    "is_default": false,
    "member_created_id": 10,
    "member_updated_id": null,
    "name": "ENTITY_TRAINING_SCHEMA",
    "project_id": 4,
    "time_created": "2025-02-05 17:20:02",
    "time_updated": null
  }
]
Schema 'ENTITY_TRAINING_SCHEMA' already exists with id: 11


## Get diffgram tasks

In [8]:
results = project.job
get_job = project.job.list(limit=10000, page_number=1)

In [9]:
jobs_with_data_index = du.get_all_tasks(project, get_job)

In [10]:
#jobs_with_data_index

## NER Schema labesl

In [11]:
schema_list = du.get_schema_list(schema_id, project, DIFFGRAM_CONFIG)
ner_schema_list = []
for labels in schema_list['labels_out']:
    #print(labels['label']['name'])
    ner_schema_list.append(labels['label']['name'])

## Populate df of all the data for whylogs to process

In [12]:
def extract_and_arrange(completed_annotations, pd_idx, pd_word, pd_schema, pd_job_id, dataset_id, pd_job_name, pd_dataset_url, job_index):
    for completed_annotation in completed_annotations:
        #print(f"{completed_annotation} ----")
        if completed_annotation in ["attribute_groups_reference", "export_info", "label_map", "readme", "label_colour_map"]:
            continue  # Skip metadata entries
        sentence_local = []
        labels_local = []
        task_url = None

        job_id = get_job[job_index["index"]]["id"]
        if job_id and auth:
            task_url = du.get_diffgram_task_url(job_id, completed_annotation, auth, DIFFGRAM_CONFIG)
            task_url = task_url.replace("dispatcher", "localhost") if task_url else "URL Not Found"

            
        #print(completed_annotations[completed_annotation]['text']['tokens']['words'])
        for idx, words in enumerate(completed_annotations[completed_annotation]['text']['tokens']['words']):
            #print(idx)
            #print(words)
            #print(f"Idx: {idx} Word: {words['value']}")
            ## find the token value
            for start in completed_annotations[completed_annotation]['instance_list']:
                if 'start_token' in start:
                    if idx == start['start_token']:
                        ## Find the schmea value
                        schema_id = completed_annotations['label_map'][str(start['label_file_id'])]
                        #print(f"Idx: {idx} {words['value']} {start['start_token']} schmea: {schema_id}")
                        pd_idx.append(idx)
                        pd_word.append(words['value'])
                        pd_schema.append(schema_id)
                        pd_job_id.append(job_index['index'])  
                        pd_job_name.append(job_index['nickname'])
                        pd_dataset_id.append(completed_annotation)
                        pd_dataset_url.append(f'<a href="{task_url}" target="_blank">View</a>')
                                # Get task URL if parameters are provided

In [13]:
files_index_in_job_total = []
pd_idx = []
pd_word = []
pd_schema = []
pd_job_id = []  # New list for job IDs
pd_job_name= []
pd_dataset_url = []
pd_dataset_id = []  # New list for dataset IDs
start_index = 50
last_index = 51 # len(jobs_with_data_index)
for job_index in jobs_with_data_index[start_index:last_index]:
    print(f"The job nickname is {job_index['nickname']} and the index is {job_index['index']}")
    results.refresh_from_dict(get_job[job_index['index']])
    completed_annotations = results.generate_export()
    files_index_in_job = []
    extract_and_arrange(completed_annotations, pd_idx, pd_word, pd_schema, pd_job_id, pd_dataset_id, pd_job_name, pd_dataset_url, job_index)
    files_index_in_job_total.append(files_index_in_job)

The job nickname is NER_train_batch_199 and the index is 50


In [14]:
# Convert to pandas DataFrame
import pandas as pd
df = pd.DataFrame({
    'dataset_id': pd_dataset_id,
    'token index': pd_idx,
    'word': pd_word,
    'schema': pd_schema,
    'job_id': pd_job_id,
    'job_name':pd_job_name,
    'task_url':pd_dataset_url
})

In [ ]:
display(HTML(df.to_html(escape=False)))

## Whylogs Constraints

In [26]:
check_b_metadata_field_count = lambda x: {
    item.value: item.est for item in x.to_summary_dict()['frequent_strings']
}.get('B-METADATA_FIELD', 0) == 7

In [27]:
check_b_metadata_value_count = lambda x: {
    item.value: item.est for item in x.to_summary_dict()['frequent_strings']
}.get('B-METADATA_VALUE', 0) == 6

In [28]:
def check_metadata_annotation_overall(metric):
    """Check if B-METADATA_VALUE count is 6 and the rest of the medata data is 0"""
    item_counts = {
        item.value: item.est for item in metric.to_summary_dict()['frequent_strings']
    }
    meta_field = item_counts.get('B-METADATA_FIELD', 0)
    meta_value = item_counts.get('B-METADATA_VALUE', 0)
    if ((meta_value ==  6) and (meta_field ==7)):
        return True
    else:
        return False

## Profile dataset using whylogs

In [29]:
import whylogs as why
from whylogs.core import DatasetProfileView

# Profile the entire dataset
profile = why.log(df)

# You can also profile by dataset_id and job_id
profiles_by_dataset = {}
for dataset_id in df['dataset_id'].unique():
    dataset_slice = df[df['dataset_id'] == dataset_id]
    profiles_by_dataset[dataset_id] = why.log(dataset_slice)

# Get schema counts
schema_counts = df.groupby(['dataset_id', 'schema']).size().reset_index(name='count')

In [124]:
def display_visualization(profile_view, constraints=None):
    viz = NotebookProfileVisualizer()
    viz.set_profiles(target_profile_view=profile_view)
    display(profile_view.to_pandas())
    display(viz.profile_summary())
    if constraints:
        display(viz.constraints_report(constraints, cell_height=300))
    display(viz.distribution_chart("schema"))

In [215]:
def check_section_annotation(profile_view):
    try:
        items = profile_view.get_column("word").get_metric("frequent_items").to_summary_dict()["frequent_strings"]
        words = {item['value']: item['est'] for item in items}
        return "section" in words and words["section"] > 0
    except:
        return False


In [225]:
import pandas as pd
import whylogs as why
from whylogs.core.constraints import ConstraintsBuilder
from whylogs.core.constraints import Constraints, ConstraintsBuilder, MetricsSelector, MetricConstraint
from whylogs.viz import NotebookProfileVisualizer
#from whylogs.core.metrics.metrics_selector import MetricsSelector
#from IPython.display import display, HTML

schema_issues = []
#schema_types = ner_schema_list
# Define all schema types to track
schema_types = [
    'B-METADATA_FIELD', 
    'B-METADATA_VALUE',
    'B-CHUNK_ID',
    'B-ACT_ID',
    'B-SECTION_NAME',
    'B-REGULATION_ID',
    'B-SECTION_ID',
    'B-SEQUENCE_ID'
]

for profile in profiles_by_dataset:
    #print(profile)
    #print(profiles_by_dataset[profile])
    whylog_data_profile = profiles_by_dataset[profile]
    profile_view = whylog_data_profile.view()
    builder = ConstraintsBuilder(dataset_profile_view=profile_view)

    ## Extract URL
    task_url_summary = profile_view.get_column('task_url').to_summary_dict()
    frequent_strings = task_url_summary.get('frequent_items/frequent_strings', [])
    if frequent_strings and len(frequent_strings) > 0:
        url_item = frequent_strings[0]
        url_value = url_item.value

    ## Extract Job Name
    job_name = profile_view.get_column('job_name').to_summary_dict()
    frequent_strings = job_name.get('frequent_items/frequent_strings', [])
    if frequent_strings and len(frequent_strings) > 0:
        job_item = frequent_strings[0]
        task_name = job_item.value

    ## Extract dataset id
    dataset_id = profile_view.get_column('dataset_id').to_summary_dict()
    frequent_strings = dataset_id.get('frequent_items/frequent_strings', [])
    if frequent_strings and len(frequent_strings) > 0:
        dataset_item = frequent_strings[0]
        file_id = dataset_item.value

    

        # ✅ Add constraint for metadata fields
    builder.add_constraint(
        MetricConstraint(
            name="B-METADATA_FIELD Count",
            condition=check_b_metadata_field_count,
            metric_selector=MetricsSelector(column_name="schema", metric_name="frequent_items"),
        )
    )
    meta_field_constraint = builder.build().validate(profile_view)

    builder.add_constraint(
        MetricConstraint(
            name="B-METADATA_VALUE Count",
            condition=check_b_metadata_value_count,
            metric_selector=MetricsSelector(column_name="schema", metric_name="frequent_items"),
        )
    )
    meta_value_constraint = builder.build().validate(profile_view)

    builder.add_constraint(
        MetricConstraint(
            name="METADATA OVERALL Count",
            condition=check_metadata_annotation_overall,
            metric_selector=MetricsSelector(column_name="schema", metric_name="frequent_items"),
        )
    )

    meta_overall_constraint = builder.build().validate(profile_view)

    constraints = builder.build()

    # Create a dictionary for the current file's results
        # Store summary row, profile_view, and visualization function
    file_result = {
        "Task #": task_name,
        "File #": file_id,
        "URL": f'{url_value}',
        "Metadata Pass": f"✔ ({meta_overall_constraint})" if meta_overall_constraint else f"❌ ({meta_overall_constraint})",
        #"profile_view": profile_view,
        #"constraints": constraints,
        "render_visualization": lambda pv=profile_view, cs=constraints: display_visualization(pv, cs)
    }

    
    freq_values_values = profile_view.get_column("word").get_metric("frequent_items").to_summary_dict()
    word_counts = {item.value: item.est for item in freq_values_values['frequent_strings']}

    section_variants = {"section", "Section", "sections", "Sections"}
    section_found = {word: count for word, count in word_counts.items() if word in section_variants}
    
    if section_found['Section'] > 2 or len(section_found) > 1:
        print("Section-like words found:", section_found)
    else:
        print("No 'section' variants found.")
        

    #Check if sections are annotated right
    builder.add_constraint(check_section_annotation(word_counts),
    )

    # Count occurrences of each schema type
    for schema_type in schema_types:
        if schema_type in item_counts: 
            count = item_counts[schema_type]
        else:
            count = 0
        # For now, set a minimum threshold of 1 for all types except the original two
        min_threshold = 5 if schema_type in ['B-METADATA_FIELD', 'B-METADATA_VALUE'] else 1
        status = f"✔ ({count})" if count >= min_threshold else f"❌ ({count})"
        file_result[f"Schema {schema_type}"] = status

    # ✅ Store results
    schema_issues.append(file_result)

    visualization = NotebookProfileVisualizer()
    visualization.set_profiles(target_profile_view=profile_view, reference_profile_view=profile_view)
    #visualization.set_profiles(target_profile_view=profile_view)

    # ✅ Validate constraints for this file
    meta_value_constraint = builder.build().validate(profile_view)
    #break

No 'section' variants found.
No 'section' variants found.
Section-like words found: {'Section': 2, 'section': 1, 'sections': 1}
Section-like words found: {'Section': 2, 'section': 1}
Section-like words found: {'section': 3, 'Section': 2}
No 'section' variants found.
No 'section' variants found.
No 'section' variants found.
No 'section' variants found.
Section-like words found: {'Section': 2, 'section': 1}
Section-like words found: {'Section': 2, 'section': 1}
No 'section' variants found.
Section-like words found: {'section': 3, 'Section': 2}
No 'section' variants found.
Section-like words found: {'Section': 2, 'section': 1}
Section-like words found: {'Section': 2, 'section': 1}
No 'section' variants found.
No 'section' variants found.
Section-like words found: {'Section': 2, 'section': 1}
No 'section' variants found.
Section-like words found: {'Section': 2, 'section': 2}
Section-like words found: {'Section': 2, 'section': 1}
Section-like words found: {'Section': 2, 'section': 2}
No 'se

In [211]:
items = profile_view.get_column("word").get_metric("frequent_items").to_summary_dict()["frequent_strings"]
for index, item in enumerate(items):
    if item.__dict__['value'] == 'section' or item.__dict__['value'] == 'Section':
        print(item.__dict__)
        print(f"Section found")
#words = {item['value']: item['est'] for item in items}
#return "section" in words and words["section"] > 0

{'value': 'Section', 'est': 2, 'upper': 2, 'lower': 2}
Section found
{'value': 'section', 'est': 1, 'upper': 1, 'lower': 1}
Section found


In [224]:
len(section_found)

1

In [171]:
# ✅ Convert to DataFrame for Display
schema_issues_df = pd.DataFrame(schema_issues)

# ✅ Display the validation summary table
display(HTML(schema_issues_df.to_html(escape=False)))

,Task #,File #,URL,Metadata Pass,render_visualization,Schema B-METADATA_FIELD,Schema B-METADATA_VALUE,Schema B-CHUNK_ID,Schema B-ACT_ID,Schema B-SECTION_NAME,Schema B-REGULATION_ID,Schema B-SECTION_ID,Schema B-SEQUENCE_ID
0,NER_train_batch_199,19123,View,✔ (True),at 0x7efe74e976a0>,✔ (7),✔ (6),❌ (0),❌ (0),❌ (0),❌ (0),❌ (0),❌ (0)
1,NER_train_batch_199,19124,View,❌ (False),at 0x7efe5721bba0>,✔ (7),❌ (0),✔ (1),✔ (1),✔ (1),✔ (1),✔ (1),✔ (1)
2,NER_train_batch_199,19125,View,✔ (True),at 0x7efe74aa9940>,✔ (7),✔ (6),❌ (0),❌ (0),❌ (0),❌ (0),❌ (0),❌ (0)
3,NER_train_batch_199,19126,View,❌ (False),at 0x7efe74a3e520>,✔ (7),❌ (0),✔ (1),✔ (1),✔ (1),✔ (1),✔ (1),✔ (1)
4,NER_train_batch_199,19127,View,✔ (True),at 0x7efe74ec1940>,✔ (7),✔ (6),❌ (0),❌ (0),❌ (0),❌ (0),❌ (0),❌ (0)
5,NER_train_batch_199,19128,View,✔ (True),at 0x7efe56e49bc0>,✔ (7),✔ (6),❌ (0),❌ (0),❌ (0),❌ (0),❌ (0),❌ (0)
6,NER_train_batch_199,19129,View,❌ (False),at 0x7efe74e97a60>,✔ (7),❌ (0),✔ (1),✔ (1),✔ (1),✔ (1),✔ (3),✔ (1)
7,NER_train_batch_199,19130,View,❌ (False),at 0x7efe564ebb00>,✔ (7),❌ (0),✔ (1),✔ (1),✔ (1),✔ (1),✔ (1),✔ (1)
8,NER_train_batch_199,19131,View,❌ (False),at 0x7efe74a3fba0>,✔ (7),❌ (0),✔ (1),✔ (1),✔ (1),✔ (1),✔ (1),✔ (1)
9,NER_train_batch_199,19132,View,❌ (False),at 0x7efe746bc400>,✔ (7),❌ (1),✔ (1),✔ (1),✔ (1),❌ (0),✔ (1),✔ (1)


In [149]:
import ipywidgets as widgets
from IPython.display import display, clear_output

dropdown = widgets.Dropdown(
    options=[(f"{entry['Task #']} - {entry['File #']}", i) for i, entry in enumerate(schema_issues)],
    description='Select File:',
    layout={'width': '60%'}
)

output = widgets.Output()

def on_change(change):
    output.clear_output()
    with output:
        selected_index = change["new"]
        if selected_index is not None:
            schema_issues[selected_index]["render_visualization"]()

dropdown.observe(on_change, names="value")

display(dropdown, output)

Dropdown(description='Select File:', layout=Layout(width='60%'), options=(('NER_train_batch_199 - 19123', 0), …

Output()

In [135]:
#profile_view.to_pandas()

In [134]:
#visualization.constraints_report(constraints, cell_height=300)

In [133]:
#visualization.profile_summary()

In [45]:
#visualization.summary_drift_report()

In [132]:
#visualization.distribution_chart("schema")

In [48]:
# visualization.double_histogram(feature_name="word")  # Only works with numbers

In [49]:
#visualization.difference_distribution_chart(feature_name=["schema"]) ## Because there is no difference between the ref and target

In [60]:
#visualization.feature_statistics(feature_name=["schema"], profile="reference")

In [127]:
#freq_values_values

In [128]:
#item_counts

In [129]:
#schema_counts

In [130]:
#ner_schema_list

## Export Visualization

In [131]:
import os
os.getcwd()
visualization.write(
    visualization.difference_distribution_chart(feature_name="schema"),
    html_file_name=os.getcwd() + "/example",
)

In [191]:
display(dropdown, output)

Dropdown(description='Select File:', index=6, layout=Layout(width='60%'), options=(('NER_train_batch_199 - 191…

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '             cardinality/est  cardinal…